In [1]:
#Importation des bibliothèques
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import math
import scipy.special


###### Ici, on importe et traite les données #########

#Importation des données
data_training = pd.read_csv('sign_mnist_train.csv').to_numpy()
data_test = pd.read_csv('test.csv').to_numpy() 

# Division entre cibles et données (entrainement)
cibles = data_training[:, 0]
images_train = data_training[:, 1:]

# Division des deux images dans les données de test
image1_test = data_test[:, :784]
image2_test = data_test[:, 785:]

#Correction de la translation + bruit sur la première image de test
image1_test = image1_test.reshape(3000, 28, 28)
image1_test[:, :, :-1] = image1_test[:, :,1:]
image1_test = image1_test.reshape(3000, (784))

# Greyscale values -> [0,1]

images_train = (images_train - np.mean(images_train, axis = 1)[:, np.newaxis])/255
image1_test = (image1_test - np.mean(image1_test, axis = 1)[:, np.newaxis])/255
image2_test = (image2_test - np.mean(image2_test, axis = 1)[:, np.newaxis])/255

# Reconstitution des tableaux pour les images
images_train = images_train.reshape((len(images_train),28,28))
image1_test = image1_test.reshape((len(image1_test),28,28))
image2_test = image2_test.reshape((len(image2_test),28,28))


In [2]:
'''
Cette partie permet définie la fonction qui prend les images et donne leur décomposition en 'mode de Krawtchouk'.
Ces modes sont analogues aux séries de Fourier.
'''

##### Fonctions utiles #######

def poch(n,k):
    prod = 1
    for i in range(k):
        prod = prod *(n+i)
    return prod

def hypergeo2f1(a,b,c,x):
    somme = 0
    if ((a !=c) or (b !=c)) :
        for i in range(int(min([-a,-b]))+1):
            somme += poch(a,i)*poch(b,i)*(x)**i /(poch(c,i)*math.factorial(i))
    else:
        for i in range(int(min([-a,-b]))+1):
            somme += poch(a,i)*(x)**i /(math.factorial(i))
    return somme

def Kraw(n,k,N):
    return ((1/2)**(N/2)) *np.sqrt( scipy.special.binom(N, n) * scipy.special.binom(N, k)) *hypergeo2f1(-n,-k,-N,2) 


## Tableau contenant les vecteurs pour chaque mode de Krawtchouk
K = np.array([[Kraw(n,k,27) for n in range(27 + 1)] for k in range(27 + 1)])

#Fonction qui prend une image et redonne sa décomposition en mode de Krawtchouk
def ImageTransform(Image, kx, ky):
    Kx = K[kx,:].reshape(1,28)
    Ky = K[ky,:].reshape(28, 1)
    filtre = Ky @ Kx
    return np.sum(filtre*Image)

In [3]:
'''
Ici, on prend les données pour chaque image et on calcule sa décomposition en mode de Krawtchouk.
L'amplitude de ces modes de Krawtchouk formeront les attributs utilisé par les modèles.
'''
images_train_kraw = np.zeros_like(images_train) # Tableau qui contiendra les amplitudes de Krawtchouk (train)
image1_test_kraw = np.zeros_like(image1_test) # Tableau qui contiendra les amplitudes de Krawtchouk (1e image, test)
image2_test_kraw = np.zeros_like(image2_test) # Tableau qui contiendra les amplitudes de Krawtchouk (2e image, test)


for choice_im in range(images_train.shape[0]):
    table = np.zeros((28,28))
    for kx in range(28):
        for ky in range(28):
            table[ky,kx] = ImageTransform( images_train[choice_im], kx, ky)
    images_train_kraw[choice_im, :, :] = table
print('Fin - (images entraînement)')

for choice_im in range(image1_test.shape[0]):
    table = np.zeros((28,28))
    for kx in range(28):
        for ky in range(28):
            table[ky,kx] = ImageTransform( image1_test[choice_im], kx, ky)
    image1_test_kraw[choice_im, :, :] = table
print('Fin - (images test 1)')

for choice_im in range(image2_test.shape[0]):
    table = np.zeros((28,28))
    for kx in range(28):
        for ky in range(28):
            table[ky,kx] = ImageTransform(image2_test[choice_im], kx, ky)
    image2_test_kraw[choice_im, :, :] = table
print('Fin - (images test 2)')


Fin - (images entraînement)
Fin - (images test 1)
Fin - (images test 2)


In [4]:
'''
À partir d'ici, on défini les classes implémentant le modèle de forêt aléatoire.

1- D'abord, on utilise et modifie les classes associés à un classifieur par arbre de décision
   utilisée dans le TP7, voir https://colab.research.google.com/drive/1djfzz7iRIVeIT7WtDZ2OXgOPaEtL1k55.

2- Ensuite, on définit une classe 'randomForest' qui combine plusieurs de ces arbres pour faire
   un modèle de forêt aléatoire.

'''

# Classifieur pour un arbre unique 

#Fonction d'aide
def histo_from_labels(labels):
    labels_list = np.unique(labels).tolist()
    labels_list.append(27)
    return np.histogram(labels, bins = labels_list)

def entropy_from_distribution(dist):
    dist = dist/np.sum(dist)
    return np.sum(- dist * np.log(dist))

def entropy_from_test(distT, distF):
    NT = np.sum(distT)
    NF = np.sum(distF)
    N = NT + NF
    return (NT/N)* entropy_from_distribution(distT) + (NF/N)* entropy_from_distribution(distF) 
    
###########  Classe pour chaque noeud  #####################

class Node():
    def __init__(self):
        self.threshold = None
        self.col = None
        self.is_leaf = None
        self.output_class = None
        self.left_child=None
        self.right_child=None
        self.real_depth = None

    def find_best_question(self, x, y):
        best_col = 0
        best_val = 0
        best_loss = np.inf

        num_cols = x.shape[1]
        valid_cols = np.arange(num_cols) # nb of features
        for col in valid_cols:
            # Calculer les valeurs intermédiaires de cette colonne ici
            sorted_indices = x[:, col].argsort()
            sorted_vals = x[sorted_indices, col]
            midpoints = [(sorted_vals[i]+sorted_vals[i+1])/2 for i in range(len(sorted_vals)-1)]

            for val in midpoints:

                test = (x[:,col] > val)
                
                coord_true = np.nonzero(test.astype(int))
                coord_false = np.nonzero(np.invert(test).astype(int))

                distT = histo_from_labels(y[coord_true])[0]
                distF = histo_from_labels(y[coord_false])[0]

                loss =  entropy_from_test(distT, distF)


                if len(coord_true) == 0 or len(coord_false) == 0:
                    continue

                if loss < best_loss:
                    best_loss = loss
                    best_col = col
                    best_val = val

        self.col = best_col
        self.threshold = best_val

    def ask_question(self, x):
        if not self.is_leaf:
            return np.nonzero((x[:, self.col] > self.threshold).astype(int))[0],np.nonzero((x[:, self.col] <= self.threshold).astype(int))[0]
        else:
            print("Error: leaf nodes cannot ask questions!")
            return False

    def predict(self):
        return self.output_class

############## Classe pour l'arbre ##########################

class DecisionTreeClassifier():
    def __init__(self, max_depth=1):
        self.max_depth = max_depth
        self.actual_depth = 0

    def create_node(self, x_subset, y_subset, depth):
        # Recursive function
        self.actual_depth = max(self.actual_depth, depth)
        node = Node()
        node.real_depth = depth
        
        histy = histo_from_labels(y_subset)
        
        
        majority_class = histy[1][np.argmax(histy[0])]
        #print('mc:', majority_class)
        majority_class_count = (y_subset == majority_class).sum()
        perfectly_classified = majority_class_count == len(y_subset)
        
        #modif
        node.output_class = majority_class
        
        if perfectly_classified or depth == self.max_depth:
            node.output_class = majority_class
            node.is_leaf = True
        else:
            node.find_best_question(x_subset,y_subset)
            node.is_leaf = False
            coord_true_sub,coord_false_sub  = node.ask_question(x_subset)
  
            node.left_child = self.create_node(x_subset[coord_true_sub],y_subset[coord_true_sub],depth+1) 
            node.right_child = self.create_node(x_subset[coord_false_sub],y_subset[coord_false_sub], depth+1) 

        return node

    def fit(self, x, y):
        self.root_node = self.create_node(x,y,depth=1)
        #print('profondeur finale:', self.actual_depth)

    def predict(self, x, cut_descend = None):
        predictions = []

        for i in range(len(x)):
            current_node = self.root_node
            x_i = x[i, :]
            done_descending_tree = False
            while not done_descending_tree:
                if current_node.is_leaf or cut_descend == current_node.real_depth:
                    predictions.append(current_node.predict())
                    done_descending_tree = True

                else:
                    if len(current_node.ask_question(np.array([x_i]))[0]) == 0 :
                        current_node = current_node.right_child
                    else:
                        current_node = current_node.left_child

        return np.array(predictions)

In [5]:
# Classifieur pour une forêt aléatoire

class randomForest():
    def __init__(self, nbarbres, nbfeatures, nbexamples, profondeurs):
        self.nbarbres = nbarbres
        self.nbfeatures = nbfeatures
        self.nbexamples = nbexamples
        self.profondeurs = profondeurs
        
    def fit(self, x, y):
        nbdata, nbattribut = x.shape
        trees = []
        subargs = []
        for tree in range(self.nbarbres):
            #print('fitting tree number:', tree)
            classifieur_arbre = DecisionTreeClassifier(max_depth = self.profondeurs)
            
            sub_ex_args = np.random.randint(nbdata, size=self.nbexamples)
            sub_att_args = np.random.randint(nbfeatures, size=self.nbfeatures)
            
            subargs.append(sub_att_args)

            subx = (x[sub_ex_args, :])[:, sub_att_args]
            suby = y[sub_ex_args]
            classifieur_arbre.fit(subx,suby)
            
            trees.append(classifieur_arbre)
        print('done training')
        self.trees = trees
        self.subargs = subargs
        
    def VecteurPredictions(self, x, fractionTrees = 1, cut_desc = None):
        vecteur = []
        for index_tree in range(int(fractionTrees*len(self.trees))):
            tree = self.trees[index_tree]
            args = self.subargs[index_tree]
            vecteur.append(tree.predict(x[:, args], cut_descend = cut_desc))
        return np.array(vecteur)
    
    def PredictionsFinals(self, x, fT = 1, cd = None):
        num_lettres = [i for i in range(27)]
        vec = self.VecteurPredictions(x,fractionTrees = fT, cut_desc = cd)
        preds = []
        for image in range(len(x)):
            histo = np.histogram(vec[:,image], bins = num_lettres)
            preds.append(num_lettres[np.argmax(histo[0])])
        return preds

In [6]:
'''
À partir d'ici, on entraîne et test le modèle.
'''
###### Division des données en entraînement et validation ##################
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images_train_kraw, cibles, test_size=0.2, random_state=42)

In [7]:
### Entraînement ###

#Grille de recherche pour l'optimisation des hyperparamètres
li_nbarbres = [10] # [10, 25, 50, 100, 150, 200]  
li_nbfeatures = [15] #[15]
li_nbexamples = [100] #[100, 200, 500, 1000]
li_profondeurs = [5,7,9,11] #[5,7,9,11]

success_rate_train = []
success_rate_valid = []

# Gridsearch
for w in li_nbarbres:
    for x in li_nbfeatures:
        for y in li_nbexamples:
            for z in li_profondeurs:
                nbarbres, nbfeatures, nbexamples, profondeurs = w,x,y,z
                print('nbarbres = ' + str(nbarbres))
                print('nbfeatures = ' + str(nbfeatures))
                print('nbexamples = ' + str(nbexamples))
                print('profondeurs = ' + str(profondeurs))
                
                model = randomForest(nbarbres, nbfeatures, nbexamples, profondeurs)
                model.fit(X_train.reshape((len(X_train), (28 )**2)), y_train)
                
                #Calcul du taux de succès d'entrainement
                predictions_train = model.PredictionsFinals(X_train.reshape((len(X_train), (28 )**2)), fT = 0.5 , cd = 5)
                success_rate_train.append(np.sum((predictions_train == y_train).astype(int))/len(predictions_train))
                
                #Calcul du taux de succès de test
                predictions_test = model.PredictionsFinals(X_test.reshape((len(X_test),(28)**2)),fT = 0.5 , cd = 5)
                success_rate_valid.append(np.sum((predictions_test == y_test).astype(int))/len(predictions_test))
                
                print('Success rate training = ' + str(success_rate_train[-1]))
                print('Success rate validation = ' + str(success_rate_valid[-1]))
                print(' ---------- ')

nbarbres = 10
nbfeatures = 15
nbexamples = 100
profondeurs = 5
done training
Success rate training = 0.14637588781642688
Success rate validation = 0.14059369877982153
 ---------- 
nbarbres = 10
nbfeatures = 15
nbexamples = 100
profondeurs = 7
done training
Success rate training = 0.15070114733199783
Success rate validation = 0.14696776543434711
 ---------- 
nbarbres = 10
nbfeatures = 15
nbexamples = 100
profondeurs = 9
done training
Success rate training = 0.15202148971043525
Success rate validation = 0.14059369877982153
 ---------- 
nbarbres = 10
nbfeatures = 15
nbexamples = 100
profondeurs = 11
done training
Success rate training = 0.17738116918594063
Success rate validation = 0.17792751775632853
 ---------- 


In [10]:
'''
Ici, on applique le meilleur modèle sur les données de test et on produit les prédictions pour Kaggle
'''

# Entraînement du modèle avec les meilleures performances en validation
best_nbarbres, best_nbfeatures, best_nbexamples, best_profondeurs = 100, 15,1000,11 #meilleurs paramètres
model = randomForest(best_nbarbres, best_nbfeatures, best_nbexamples, best_profondeurs)
model.fit(X_train.reshape((len(X_train), (28 )**2)), y_train)


### Fonction qui somme les ASCII de deux images
# Nombre entre 0 et 26 vers charactère ASCII
def SumASCII(arr1, arr2):
    
    #Somme des ASCII des deux charactères et retrait des 65 pour int [65,122]
    arrSUM = np.array(arr1) + np.array(arr2) + 65
    
    #Somme vers charactères
    char_list = []
    for num in arrSUM:
        char_list.append(chr(num))
        
    return np.array(char_list)

#Predictions pour la première image de chaque paire
predictions_test1 =  model.PredictionsFinals(image1_test_kraw.reshape((len(image1_test_kraw), (28 )**2)), cd = 10)

#Predictions pour la seconde image de chaque paire
predictions_test2 = model.PredictionsFinals(image2_test_kraw.reshape((len(image2_test_kraw), (28 )**2)), cd = 10)

#Prediction du charactère latin correspondant à la somme ASCII
predictions_char = SumASCII(predictions_test1,predictions_test2)

###############################################################################
# Création du fichier csv de prédictions pour Kaggle

test_pred = np.c_[ np.reshape(np.array(range(len(predictions_char ))), (len(predictions_char ), 1)),np.reshape(np.array(predictions_char ), (len(predictions_char), 1))]
df = pd.DataFrame(test_pred, columns = ['id', 'Label'])
df.to_csv("pred_rf.csv", index = False)
